In [1]:
!pip install tensorflow

In [2]:
!pip install rarfile
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
import rarfile
import os


rarPath1 = '/content/TIRES.rar'
rarPath2 = '/content/random.rar'

extractPath1 = '/content/'
extractPath2 = '/content/Random'

with rarfile.RarFile(rarPath1) as rf:
    rf.extractall(extractPath1)

with rarfile.RarFile(rarPath2) as rf:
    rf.extractall(extractPath2)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

trainingPath = '/content/TIRES'
testingPath = '/content/Random'

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    trainingPath,
    target_size=(380, 380),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    trainingPath,
    target_size=(380, 380),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    testingPath,
    target_size=(380, 380),
    batch_size=32,
    class_mode=None,  # No labels
    shuffle=False
)

print(f"Found {train_generator.samples} images belonging to {train_generator.num_classes} classes in training set.")
print(f"Found {validation_generator.samples} images belonging to {validation_generator.num_classes} classes in validation set.")
print(f"Found {test_generator.samples} images in testing set.")



Found 2402 images belonging to 4 classes.
Found 598 images belonging to 4 classes.
Found 2699 images belonging to 1 classes.
Found 2402 images belonging to 4 classes in training set.
Found 598 images belonging to 4 classes in validation set.
Found 2699 images in testing set.


In [7]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the EfficientNetB4 model
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(380, 380, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size

model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=50
)


Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 293s 2s/step - accuracy: 0.9193 - loss: 0.2719 - val_accuracy: 0.4062 - val_loss: 4.4985
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 7s 87ms/step - accuracy: 1.0000 - loss: 0.0108 - val_accuracy: 0.3182 - val_loss: 5.1295
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.9839 - loss: 0.0878 - val_accuracy: 0.4062 - val_loss: 3.3103
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.9688 - loss: 0.1006 - val_accuracy: 0.3182 - val_loss: 4.5384
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.9938 - loss: 0.0227 - val_accuracy: 0.3872 - val_loss: 1.9451
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 0.0043 - val_accuracy: 0.3636 - val_loss: 1.8423
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.9928 - loss: 0.0533 - val_accuracy: 0.2483 - val_loss: 2.4389
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 0.0054 - val_accuracy: 0.0909 - val_

In [15]:
labels = (train_generator.class_indices)

labels = dict((v, k) for k, v in labels.items())

filenames = test_generator.filenames

predictions = model.predict(test_generator, steps=(test_generator.samples // test_generator.batch_size) + 1)

predicted_class_indices = predictions.argmax(axis=-1)

if len(predicted_class_indices) != len(filenames):
    print(f"Number of predictions ({len(predicted_class_indices)}) does not match number of filenames ({len(filenames)}).")
else:
    labelsDict = {filename: labels[predicted_class_indices[i]] for i, filename in enumerate(filenames)}

    print(labelsDict)



85/85 ━━━━━━━━━━━━━━━━━━━━ 40s 479ms/step
{'random/14038843825.JPG': 'MultiTires', 'random/140422934160.JPG': 'MultiTires', 'random/140423293635.JPG': 'side', 'random/140424023629.JPG': 'MultiTires', 'random/140424790626.JPG': 'MultiTires', 'random/140441004586.JPG': 'MultiTires', 'random/140483604855.JPG': 'MultiTires', 'random/140484913180.JPG': 'MultiTires', 'random/140501173862.JPG': 'MultiTires', 'random/140510444142.JPG': 'side', 'random/140510482877.JPG': 'side', 'random/140510673837.JPG': 'MultiTires', 'random/140544067164.JPG': 'MultiTires', 'random/140545042860.JPG': 'MultiTires', 'random/140596269813.JPG': 'MultiTires', 'random/140605904483.JPG': 'MultiTires', 'random/140630795691.JPG': 'side', 'random/140631140781.JPG': 'MultiTires', 'random/140665219438.JPG': 'side', 'random/140665689016.JPG': 'side', 'random/14066576948.JPG': 'MultiTires', 'random/140691156330.JPG': 'MultiTires', 'random/14069120982.JPG': 'MultiTires', 'random/140699513911.JPG': 'MultiTires', 'random/1407